In [1]:
from keras.layers import Conv2D, Conv2DTranspose, UpSampling2D
from keras.layers import Activation, Dense, Dropout, Flatten, InputLayer
from keras.layers.normalization import BatchNormalization
from keras.callbacks import TensorBoard
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray
from skimage.io import imsave
import numpy as np
import os
import random
import tensorflow as tf

Using TensorFlow backend.


In [6]:
# change the path to your train pictures file
train_path = './Train/';

X = []
for filename in os.listdir(train_path):
    X.append(img_to_array(load_img(train_path + filename)))
X = np.array(X, dtype=float)

# Set up train and test data
split = int(0.90*len(X))
print(split)
Xtrain = X[:split]
Xtrain = 1.0/255*Xtrain

144


In [7]:
model = Sequential()
model.add(InputLayer(input_shape=(96, 96, 1)))
model.add(Conv2D(24, (3, 3), activation='relu', padding='same'))              
model.add(Conv2D(24, (3, 3), activation='relu', padding='same', strides=2))   
model.add(Conv2D(48, (3, 3), activation='relu', padding='same'))              
model.add(Conv2D(48, (3, 3), activation='relu', padding='same', strides=2))   
model.add(Conv2D(96, (3, 3), activation='sigmoid', padding='same'))              
model.add(Conv2D(96, (3, 3), activation='sigmoid', padding='same', strides=2))   
model.add(Conv2D(192, (3, 3), activation='sigmoid', padding='same'))             
model.add(Conv2D(96, (3, 3), activation='sigmoid', padding='same'))              
model.add(Conv2D(48, (3, 3), activation='sigmoid', padding='same'))              
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(24, (3, 3), activation='sigmoid', padding='same'))              
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(12, (3, 3), activation='sigmoid', padding='same'))              
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))               
model.add(UpSampling2D((2, 2)))
model.compile(optimizer='rmsprop', loss='mse')

In [8]:
# Image transformer
datagen = ImageDataGenerator(
        shear_range=0.2,
        zoom_range=0.2,
        rotation_range=20,
        horizontal_flip=True)

# Generate training data
batch_size = 15
def image_a_b_gen(batch_size):
    for batch in datagen.flow(Xtrain, batch_size=batch_size):
        lab_batch = rgb2lab(batch)
        X_batch = lab_batch[:,:,:,0]
        Y_batch = lab_batch[:,:,:,1:] / 128
        yield (X_batch.reshape(X_batch.shape+(1,)), Y_batch)

print(batch_size * split)

2160


In [9]:
# steps per epoc - whole dataset / batch_size
model.fit_generator(image_a_b_gen(batch_size), epochs=15, steps_per_epoch= split)

Epoch 1/15
144/144 [==============================] - 102s 711ms/step - loss: 0.0449
Epoch 2/15
144/144 [==============================] - 104s 722ms/step - loss: 0.0343
Epoch 3/15
144/144 [==============================] - 103s 716ms/step - loss: 0.0332
Epoch 4/15
144/144 [==============================] - 101s 703ms/step - loss: 0.0325
Epoch 5/15
144/144 [==============================] - 101s 703ms/step - loss: 0.0323
Epoch 6/15
144/144 [==============================] - 101s 703ms/step - loss: 0.0324
Epoch 7/15
144/144 [==============================] - 101s 699ms/step - loss: 0.0325
Epoch 8/15
144/144 [==============================] - 101s 699ms/step - loss: 0.0323
Epoch 9/15
144/144 [==============================] - 101s 700ms/step - loss: 0.0321
Epoch 10/15
144/144 [==============================] - 100s 696ms/step - loss: 0.0323
Epoch 11/15
144/144 [==============================] - 102s 707ms/step - loss: 0.0322
Epoch 12/15
144/144 [==============================] - 101s 698

In [10]:
Xtest = rgb2lab(1.0/255*X[split:])[:,:,:,0]
Xtest = Xtest.reshape(Xtest.shape+(1,))
Ytest = rgb2lab(1.0/255*X[split:])[:,:,:,1:]
Ytest = Ytest / 128
print(model.evaluate(Xtest, Ytest, batch_size=batch_size))

16/16 [==============================] - 0s 18ms/step
0.0152834173641


In [11]:
color_me = []
test_path = './Test/'
for filename in os.listdir(test_path):
    color_me.append(img_to_array(load_img(test_path+filename)))
color_me = np.array(color_me, dtype=float)
color_me = rgb2lab(1.0/255*color_me)[:,:,:,0]
color_me = color_me.reshape(color_me.shape+(1,))

# Test model
output = model.predict(color_me)
output = output * 128

# Output colorizations
for i in range(len(output)):
    cur = np.zeros((96, 96, 3))
    cur[:,:,0] = color_me[i][:,:,0]
    cur[:,:,1:] = output[i]
    imsave("result/img_"+str(i)+".png", lab2rgb(cur))

c:\users\tsofan\appdata\local\conda\conda\envs\tensorflow\lib\site-packages\skimage\util\dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))
